# QLORA

In [ ]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets   trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 4.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import torch
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from trl import SFTTrainer
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [ ]:
model_name='microsoft/phi-2'
device_map = {"": 0}
original_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

In [ ]:
huggingface_dataset_name = "stanfordnlp/snli"
dataset = load_dataset(huggingface_dataset_name)

README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/412k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/413k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/19.6M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/550152 [00:00<?, ? examples/s]

In [ ]:
import os
# disable Weights and Biases
os.environ['WANDB_DISABLED']="true"

In [ ]:
%%time
from transformers import set_seed
seed = 42
set_seed(seed)

CPU times: user 5.04 ms, sys: 1.48 ms, total: 6.52 ms
Wall time: 17.8 ms


In [ ]:
def gen(model,prompt,max_length):
  # Tokenize the input
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids

  # Generate a response
  output = model.generate(input_ids, max_length=max_length, num_return_sequences=1)

  # Decode the response
  response = tokenizer.decode(output[0], skip_special_tokens=True)
  return response

In [ ]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters
    :param sample: Sample dictionnary
    """
    INTRO_BLURB = ""
    INSTRUCTION_KEY = "### | "
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"
    premise = sample["premise"]
    hypothesis = sample["hypothesis"]
    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"Premise: {premise}\nHypohtesis: {hypothesis}\n  Choose the correct option: 0 - entailment, 1 - neutral, 2 - contradiction-"
    response = f"{RESPONSE_KEY}\n{sample['label']}"
    end = f"{END_KEY}"

    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt

    return sample

In [ ]:
from functools import partial

# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def get_max_length(model):
    conf = model.config
    max_length = None

    length_settings = ["n_positions", "max_position_embeddings", "seq_length"]
    index = 0

    while index < len(length_settings):
        max_length = getattr(model.config, length_settings[index], None)
        if max_length:
            print(f"Found max length: {max_length}")
            break
        index += 1

    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int,seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """

    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)#, batched=True)

    # Apply preprocessing to each batch of the dataset & and remove 'instruction', 'context', 'response', 'category' fields
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        remove_columns=['premise','hypothesis','label'],
        batched=True,
    )

    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset

In [ ]:
from datasets import Dataset, DatasetDict
# Define the number of samples and create an empty list to store the selected samples
total_samples = len(dataset["train"])
selected_samples_train = []
print(total_samples)
# Loop through every 1000 samples and pick the 550th one
for i in range(0, total_samples, 1000):
    index = i + 550 - 1  # 0-indexed, so 550th is at 549th position
    if index < len(dataset["train"]):  # Check if the index is within bounds
        selected_samples_train.append(dataset["train"][index])

train_dataset = Dataset.from_list(selected_samples_train)

total_samples = len(dataset["validation"])
selected_samples_validation = []
print(total_samples)
# Loop through every 1000 samples and pick the 550th one
for i in range(0, total_samples, 100):
    index = i + 100 - 1  # 0-indexed, so 100th is at 549th position
    if index < len(dataset["validation"]):  # Check if the index is within bounds
        selected_samples_validation.append(dataset["validation"][index])

validation_dataset = Dataset.from_list(selected_samples_validation)

total_samples = len(dataset["test"])
selected_samples_test = []
print(total_samples)
# Loop through every 1000 samples and pick the 550th one
for i in range(0, total_samples, 100):
    index = i + 100 - 1  # 0-indexed, so 100th is at 549th position
    if index < len(dataset["test"]):  # Check if the index is within bounds
        selected_samples_test.append(dataset["test"][index])

test_dataset = Dataset.from_list(selected_samples_test)

dataset_final = DatasetDict({"train": train_dataset,"validation": validation_dataset,"test":test_dataset})

550152
10000
10000


In [ ]:
## Pre-process dataset
max_length = get_max_length(original_model)
print(max_length)

train_dataset = preprocess_dataset(tokenizer, max_length,seed, dataset_final['train'])
eval_dataset = preprocess_dataset(tokenizer, max_length,seed, dataset_final['validation'])

Found max lenth: 2048
2048
Preprocessing dataset...


Map:   0%|          | 0/550 [00:00<?, ? examples/s]

Map:   0%|          | 0/550 [00:00<?, ? examples/s]

Filter:   0%|          | 0/550 [00:00<?, ? examples/s]

Preprocessing dataset...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

config = LoraConfig(
    r=32, #Rank
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

# 1 - Enabling gradient checkpointing to reduce

original_model = prepare_model_for_kbit_training(original_model)

original_model.gradient_checkpointing_enable()

peft_model = get_peft_model(original_model, config)

In [ ]:
# Count the total parameters
total_params = sum(p.numel() for p in peft_model.parameters())
print(f"Total Parameters: {total_params:,}")

# Count the trainable parameters
trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
print(f"Trainable Parameters: {trainable_params:,}")

# Optionally, you can also calculate the percentage of trainable parameters
trainable_percentage = (trainable_params / total_params) * 100
print(f"Percentage of Trainable Parameters: {trainable_percentage:.2f}%")

Total Parameters: 1,563,335,680
Trainable Parameters: 41,943,040
Percentage of Trainable Parameters: 2.68%


In [ ]:
output_dir = f'./peft-NLI'
import transformers

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="epoch",
    save_steps=25,
    evaluation_strategy="epoch",
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
    num_train_epochs=5
)

peft_model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
peft_trainer.train()

Epoch,Training Loss,Validation Loss
0,0.921200,0.926452
2,0.775000,0.934475
4,0.577000,1.024154


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


TrainOutput(global_step=685, training_loss=0.7620080397946991, metrics={'train_runtime': 1193.946, 'train_samples_per_second': 2.303, 'train_steps_per_second': 0.574, 'total_flos': 3136681165762560.0, 'train_loss': 0.7620080397946991, 'epoch': 4.9818181818181815})

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id = "microsoft/phi-2"
base_model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id = "microsoft/phi-2"
base_model_2 = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "/content/peft-NLI/checkpoint-685",torch_dtype=torch.float16,is_trainable=False)

In [ ]:
base_model_2

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiSdpaAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_la

In [ ]:
from transformers import set_seed
set_seed(seed)

index = 5
premise = dataset_final['test'][index]['premise']
hypothesis = dataset_final['test'][index]['hypothesis']
label = dataset_final['test'][index]['label']

formatted_prompt = f"Instruct: Perform NLI task  \nPremise:{premise}\nHypothesis:{hypothesis} \nOutput:\n Choose the correct option - 0 - entailment, 1 - neutral, 2 - contradiction-"

peft_model_res = gen(ft_model,formatted_prompt,100,)
print(peft_model_res)
peft_model_output = peft_model_res.split('Output:\n')[1]
#print(peft_model_output)
prefix, success, result = peft_model_output.partition('###')


print(f'INPUT PROMPT:\n{formatted_prompt}')
print(f'BASELINE HUMAN LABEL:\n{label}\n')
print(f'PEFT MODEL:\n{prefix}')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Instruct: Perform NLI task    
Premise:A woman wearing jeans is walking down the street.
Hypothesis:A woman is walking towards a bus stop. 
Output:
 
      Choose the correct option: 0 - entailment, 1 - neutral, 2 - contradiction-

Output:
1

End of Output

Premise: A woman wearing jeans is walking towards a bus stop.
Hypohtesis: A woman wearing jeans is
INPUT PROMPT:
Instruct: Perform NLI task  
Premise:A woman wearing jeans is walking down the street.
Hypothesis:A woman is walking towards a bus stop. 
Output:
 
BASELINE HUMAN LABEL:
1

PEFT MODEL:
 
      Choose the correct option: 0 - entailment, 1 - neutral, 2 - contradiction-




In [ ]:

labels_true = []
labels_predicted_original = []
labels_predicted_finetuned = []

index = 0
while index < 30:
    premise = dataset_final['test'][index]['premise']
    hypothesis = dataset_final['test'][index]['hypothesis']
    label_now = dataset_final['test'][index]['label']

    formatted_prompt = f"### Perform NLI Task \nPremise:{premise}\nHypothesis:{hypothesis}\n Choose the correct option - 0 - entailment, 1 - neutral, 2 - contradiction-\n ###Output:\n"

    peft_model_res = gen(ft_model, formatted_prompt, 100)
    peft_model_output = peft_model_res.split('Output:\n')[1]

    peft_prefix, peft_success, peft_result = peft_model_output.partition('###')

    base_model_res = gen(base_model_2, formatted_prompt, 100)
    base_model_output = base_model_res.split('Output:\n')[1]

    base_prefix, base_success, base_result = base_model_output.partition('###')

    labels_true.append(label_now)
    labels_predicted_original.append(base_prefix.strip())
    labels_predicted_finetuned.append(peft_prefix.strip())
    print(label_now, base_prefix.strip(), peft_prefix.strip())

    if !(str(label_now) == peft_prefix.strip()[0]):
        if !(str(label_now) == base_prefix.strip()[0]):
            print("CORRECT LABEL-", label_now)
            print("\n")
            print("BASE MODEL OUTPUT:", base_model_res)
            print("\n")
            print("QLORA OUTPUT:", peft_model_res)

    index += 1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention

1 0 1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


2 0 2


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

0 0 0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


1 ```python
# Solution
premise = "A team of surgeons operate on a female patient."
hypothesis = "The woman has cancer."

if premise == "A team of surgeons operate on 1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


2 0 2


In [ ]:
# sum = 0
# for index in range(100):
#   if(str(labels_true[index])==labels_predicted_finetuned[index][0]):
#     sum+=1

sum = 0
index = 0

while index < 100:
    if !(str(labels_true[index]) != labels_predicted_finetuned[index][0]):
        sum += 1
    index += 1


In [ ]:
sum

88

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mv peft-NLI /content/drive/My\ Drive/peft-NLI